<a href="https://colab.research.google.com/github/fukagai-takuya/gifu-ai/blob/main/gifu-ai-2025-08-24/Ecco-Trial-2025-07-30-removed-ouputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. ecco をインストールします。2025年7月30日に下記のコマンドでインストールした ecco は version 0.1.2 でした。

```
!pip install git+https://github.com/jalammar/ecco.git
```

このコマンドを実行すると下記のモジュールがインストールされます。
```
captum-0.8.0 ecco-0.1.2 numpy-1.26.4 nvidia-cublas-cu12-12.4.5.8 nvidia-cuda-cupti-cu12-12.4.127 nvidia-cuda-nvrtc-cu12-12.4.127 nvidia-cuda-runtime-cu12-12.4.127 nvidia-cudnn-cu12-9.1.0.70 nvidia-cufft-cu12-11.2.1.3 nvidia-curand-cu12-10.3.5.147 nvidia-cusolver-cu12-11.6.1.9 nvidia-cusparse-cu12-12.3.1.170 nvidia-nvjitlink-cu12-12.4.127
```

下記のようなエラーメッセージが表示され、インストールした numpy-1.26.4 を使用するため、Google Colab の Runtime を再起動する必要がありました。

```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
Successfully installed captum-0.8.0 ecco-0.1.2 numpy-1.26.4 nvidia-cublas-cu12-12.4.5.8 nvidia-cuda-cupti-cu12-12.4.127 nvidia-cuda-nvrtc-cu12-12.4.127 nvidia-cuda-runtime-cu12-12.4.127 nvidia-cudnn-cu12-9.1.0.70 nvidia-cufft-cu12-11.2.1.3 nvidia-curand-cu12-10.3.5.147 nvidia-cusolver-cu12-11.6.1.9 nvidia-cusparse-cu12-12.3.1.170 nvidia-nvjitlink-cu12-12.4.127
WARNING: The following packages were previously imported in this runtime:
  [numpy]
You must restart the runtime in order to use newly installed versions.
```

In [ ]:
!pip install git+https://github.com/jalammar/ecco.git

2. transfomers 4.24.0 をインストールします。

最新の transformers のモジュールでは ecco 0.1.2 を使用した以降のコードセルのスクリプトを実行することができなかったため、transfomers 4.24.0 をインストールします。

In [ ]:
!pip install transformers==4.24.0

3. 下記のコードセルを実行します。ecco を使用して gpt2 が予測した Token がどの入力 Token の影響をより強く受けていたかを色で表示します。

濃い色の Token がより強い影響を与えたことを表しています。

下記のページの Input Saliency に関するスクリプトになります。

https://jalammar.github.io/explaining-transformers/

バックプロパゲーションで計算される各入力 Token の勾配と、各入力 Token のベクトルの積を要素ごとに取り、その二乗和を影響度の強さとしています。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2')
prompt="Heathrow airport is located in the city of"

output = lm.generate(prompt, generate=1, do_sample=False, attribution=['ig'])
output.primary_attributions(attr_method='ig')

4. 一つ前のコードセルと同じ Input Saliency を表します。こちらは入力 Token の影響の大きさを数値で表します。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2')
prompt="Heathrow airport is located in the city of"

output = lm.generate(prompt, generate=1, do_sample=False, attribution=['ig'])
output.primary_attributions(attr_method='ig', style="detailed")

5. GPT2 small に複数の Token を生成させ、各 Token の生成に寄与した Token の影響力を数値で表したインタラクティブな図を出力します。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2')
prompt="In summer, when I hear the sound of cicadas,"

output = lm.generate(prompt, generate=10, do_sample=True, attribution=['ig'])
output.primary_attributions(attr_method='ig', style="detailed")

6. 下記のページに記載された、FFNN layers のニューロンの活動を NMS (Non-negative Matrix Factorization) で分析する処理に対応するスクリプトになります。

https://jalammar.github.io/explaining-transformers/

gpt2 をモデルに用いた場合は出力が文字化けしてしまったため、こちらのコードセルでは DistilBERT と BERT を使用しました。

入力として使用した英文は私の Web ページから抜粋した文章になります。

6.1. DistilBERT の全ての層の FFNN 層を対象とした場合

In [ ]:
import ecco
lm = ecco.from_pretrained('distilbert-base-uncased', activations=True)
text = 'Other species I saw on foot were Japanese grass lizards moving along the edges of the road, horseflies or carpenter bees flying near flowers with low buzzing sounds, small birds that I don’t see very often, and crows. I also heard a low buzzing sound coming from the high voltage lines. I thought the horseflies and carpenter bees, which fly with a low buzzing sound for their size, might alert humans and other animals with their sound.'
inputs = lm.tokenizer([text], return_tensors="pt")
output = lm(inputs)
nmf_1 = output.run_nmf(n_components=8)
nmf_1.explore()

6.2. DistilBERT の最初の層の FFNN 層を対象とした場合

In [ ]:
import ecco
lm = ecco.from_pretrained('distilbert-base-uncased', activations=True)
text = 'Other species I saw on foot were Japanese grass lizards moving along the edges of the road, horseflies or carpenter bees flying near flowers with low buzzing sounds, small birds that I don’t see very often, and crows. I also heard a low buzzing sound coming from the high voltage lines. I thought the horseflies and carpenter bees, which fly with a low buzzing sound for their size, might alert humans and other animals with their sound.'
inputs = lm.tokenizer([text], return_tensors="pt")
output = lm(inputs)
nmf_1 = output.run_nmf(n_components=8, from_layer=0, to_layer=1)
nmf_1.explore()

6.3. BERT の全ての層の FFNN 層を対象とした場合

In [ ]:
import ecco
lm = ecco.from_pretrained('bert-base-uncased', activations=True)
text = '''Other species I saw on foot were Japanese grass lizards moving along the edges of the road, horseflies or carpenter bees flying near flowers with low buzzing sounds, small birds that I don’t see very often, and crows. I also heard a low buzzing sound coming from the high voltage lines. I thought the horseflies and carpenter bees, which fly with a low buzzing sound for their size, might alert humans and other animals with their sound.'''
inputs = lm.tokenizer([text], return_tensors="pt")
output = lm(inputs)
nmf_1 = output.run_nmf(n_components=8)
nmf_1.explore()

7. 下記のページの各層の Hidden States の分析についてのコードセルになります。

https://jalammar.github.io/hidden-states/

7.1. gpt2 の最終層から出力された次の Token のベクトルを調べ、最もスコアが高かった Token から順に 10 の Token の候補を出力しています。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2', verbose=False)
prompt=""" Heathrow airport is located in the city of"""

output = lm.generate(prompt, generate=1, do_sample=False)
output.layer_predictions(position=9, layer=11)

7.2. gpt2 の12の各層から出力された Token ベクトルを調べ、最もスコアが高かった Token から順に 10 の Token の候補を出力しています。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2', verbose=False)
prompt=""" Heathrow airport is located in the city of"""

output = lm.generate(prompt, generate=1, do_sample=False)
output.layer_predictions(position=9)

7.3. "Heathrow airport is located in the city of" の次に来る Token が GPT-2 small の各レイヤーを経た後、何番目の候補となっていたかを表す図を表示しています。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2')
prompt="Heathrow airport is located in the city of"

output = lm.generate(prompt, generate=1, do_sample=False)
output.rankings()

7.4. "Good afternoon. " に続く 10 の Token が GPT-2 small の各レイヤーを経た後、何番目の候補となっていたかを表す図を表示しています。

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2')
prompt="Good afternoon. "

output = lm.generate(prompt, generate=10, do_sample=False)
output.rankings()

7.5. 一つ前の 7.4. と同様、"Good afternoon. " に続く 10 の Token が GPT-2 small の各レイヤーを経た後、何番目の候補となっていたかを表す図を表示しています。こちらは 7.4. とは異なり最もスコアが高い候補以外もサンプルするようにしています。(do_sample=True)

In [ ]:
import ecco
lm = ecco.from_pretrained('gpt2')
prompt="Good afternoon. "

output = lm.generate(prompt, generate=10, do_sample=True)
output.rankings()